В этом задании будем улучшать нашу систему классов из задания прошлой лекции
(Student, Teacher, Homework)
Советую обратить внимание на defaultdict из модуля collection для
использования как общую переменную


1. Как то не правильно, что после do_homework мы возвращаем все тот же
объект - будем возвращать какой-то результат работы (HomeworkResult)

HomeworkResult принимает объект автора задания, принимает исходное задание
и его решение в виде строки

Атрибуты:
    homework - для объекта Homework, если передан не этот класс -  выкинуть подходящие по смыслу исключение с сообщением: 'You gave a not Homework object'
    

    solution - хранит решение ДЗ как строку
    author - хранит объект Student
    created - c точной датой и временем создания

2. Если задание уже просрочено хотелось бы видеть исключение при do_homework,
а не просто принт 'You are late'.
Поднимайте исключение DeadlineError с сообщением 'You are late' вместо print.

3. Student и Teacher имеют одинаковые по смыслу атрибуты
(last_name, first_name) - избавиться от дублирования с помощью наследования

4.
Teacher
Атрибут:
    homework_done - структура с интерфейсом как в словаря, сюда поподают все
    HomeworkResult после успешного прохождения check_homework
    (нужно гаранитровать остутствие повторяющихся результатов по каждому
    заданию), группировать по экземплярам Homework.
    Общий для всех учителей. Вариант ипользования смотри в блоке if __main__...
Методы:
    check_homework - принимает экземпляр HomeworkResult и возвращает True если
    ответ студента больше 5 символов, так же при успешной проверке добавить в
    homework_done.
    Если меньше 5 символов - никуда не добавлять и вернуть False.

    reset_results - если передать экземпряр Homework - удаляет только
    результаты этого задания из homework_done, если ничего не передавать,
    то полностью обнулит homework_done.

PEP8 соблюдать строго.
Всем перечисленным выше атрибутам и методам классов сохранить названия.
К названием остальных переменных, классов и тд. подходить ответственно -
давать логичные подходящие имена.

In [96]:
#кастомные Exceptions
class NotRightClass(Exception):
    def __init__(self, given_class, need_class):
        super().__init__(f'You gave a {given_class} class, but need {need_class}!')
        
class DeadlineError(Exception):
    def __init__(self):
        super().__init__(f'You are late!')


In [97]:
import datetime
import inspect

class Homework:
    def __init__(self, text, deadline):
        self.text = text
        self.deadline = datetime.timedelta(days=deadline)
        self.created = datetime.datetime.now()
    def is_active(self):
        if datetime.datetime.now() > self.created + self.deadline:
            return False
        else: 
            return True
#результат Homework
class HomeworkResult:
    def __init__(self, author, homework, solution):
        #возвращает класс переданного объекта и класс, который нужен
        if not isinstance(homework, Homework):
            raise NotRightClass(type(homework).__name__, "Homework")
        if not isinstance(author, Student):
            raise NotRightClass(type(author).__name__, "Student")
        self.author = author
        self.homework = homework
        self.solution_text = solution
        self.created = datetime.datetime.now()


In [98]:
from collections import defaultdict

#родительский класс Человек
class Human:
    def __init__(self, first_name, last_name):
        self.last_name = last_name
        self.first_name = first_name

#дочерний класс Студент
class Student(Human):
    def do_homework(self, homework, solution):
        if not homework.is_active():
            raise DeadlineError()
        else:
            print("You are good - "+solution)
            return HomeworkResult(self, homework, solution)

#Дочерний класс Учитель
class Teacher(Human):
    homework_done = {} #для всех учителей общий homework_done
    def __init__(self, first_name, last_name):
        super().__init__(first_name, last_name)
    @staticmethod
    def create_homework(text, deadline):
        return Homework(text, deadline)
    
    def check_homework(self, homework_result):
        if len(homework_result.solution_text) < 5:
            return False
        else:
            #если нету результатов по какой-то работе, она создается в словаре, а только потом туда кидаются результаты
            if homework_result.homework not in self.homework_done:
                self.homework_done[homework_result.homework] = []
            self.homework_done[homework_result.homework].append(homework_result)
            return True
    @staticmethod
    def reset_results(homework=None):
        if homework is None:
            Teacher.homework_done.clear()
        else:
            Teacher.homework_done[homework].clear()

In [99]:
import datetime
from collections import defaultdict

opp_teacher = Teacher('Daniil', 'Shadrin')
advanced_python_teacher = Teacher('Aleksandr', 'Smetanin')

lazy_student = Student('Roman', 'Petrov')
good_student = Student('Lev', 'Sokolov')

oop_hw = opp_teacher.create_homework('Learn OOP', 0)
docs_hw = opp_teacher.create_homework('Read docs', 5)

result_1 = good_student.do_homework(oop_hw, 'I have done this hw')
result_2 = good_student.do_homework(docs_hw, 'I have done this hw too')
result_3 = lazy_student.do_homework(docs_hw, 'done')
try:
    result_4 = HomeworkResult(good_student, "fff", "Solution")
except Exception:
    print('There was an exception here')

You are good - I have done this hw
You are good - I have done this hw too
You are good - done
There was an exception here


In [100]:
opp_teacher.check_homework(result_1)
temp_1 = opp_teacher.homework_done

advanced_python_teacher.check_homework(result_1)
temp_2 = Teacher.homework_done
assert temp_1 == temp_2

opp_teacher.check_homework(result_2)
opp_teacher.check_homework(result_3)

print(Teacher.homework_done[oop_hw])
Teacher.reset_results()

[<__main__.HomeworkResult object at 0x000001E5DA1D4100>, <__main__.HomeworkResult object at 0x000001E5DA1D4100>]
